In [1]:
import os
import re

import django
import fs.path
import numpy as np
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
from plotter_utils import modeldict

In [2]:
for spec in ZSpec.objects.all():
    spec.delete()

In [3]:
input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/zcam")
output_fs = OSFS('.')
output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [4]:
def reg_seq_id(seq_id_str):
    seq_id_str = seq_id_str.replace(" ", "")
    return 'zcam' + format(int(seq_id_str.split('zcam')[1]), "0>5d")

metaframe = pd.read_csv(input_fs.getsyspath('Metadata-marslab.csv'))
metaframe.columns = [column.lower() for column in metaframe.columns]
# add NaNs back in so we can programmatically delete them
metaframe.replace('-', np.nan, inplace=True)
# we're turning these to ints when we ingest them,
# but python doesn't like statements like int('3.0'),
# so turn to float as an intermediate step
for column in [
    'sol', 'site', 'drive', 'rover_elevation', 'target_elevation', 'tau',
    'focal_distance', 'incidence_angle', 'emission_angle',
    'phase_angle', 'l_s', 'site', 'drive', 'lat', 'lon', 'odometry',
]:
    metaframe[column] = metaframe[column].astype('float')
metaframe['ltst'] = metaframe['ltst'].astype('datetime64')
metaframe['seq_id'] = metaframe['seq_id'].map(reg_seq_id)
metaframe.loc[
    metaframe['sclk'].notna(), 'sclk'
] = metaframe.loc[metaframe['sclk'].notna(), 'sclk'].astype(float).astype(int)

In [5]:
temp_images = [
    image for image in input_fs.listdir('temp_images')
]

In [7]:
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('-marslab.csv') and file.startswith('Sol'))
]
for spec_file in spec_files:
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    observation = metaframe.loc[metaframe['seq_id'] == frame['seq_id'][0]]
    # hacky hacky -- but temporary
    if len(observation) > 1: 
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        observation = observation.loc[observation['name'].str.contains(str(pointing))]
    assert len(observation == 1)
    for _, row in frame.iterrows():
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in ZSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])
        # because we're manually associating
        row.drop(['sol', 'seq_id', 'instrument'], inplace=True)
        metadata = dict(row) | observation.iloc[0].dropna().to_dict() | {
            'filename': spec_file
        }
        spectrum = ZSpec(**metadata)
        spectrum.clean()
        spectrum.save()

In [8]:
len(ZSpec.objects.all())

111

In [10]:
output_image_dir

'/home/michael/Desktop/multidex_flat_branch/multidex/static_in_pro/our_static/img'

In [13]:
spec.seq_id

'zcam03106'

In [14]:
os.listdir(output_image_dir)

['roi_full', 'roi_browse']

In [18]:
spec.filename

'Sol33_zcam03106_Ahyeeh_Pointing2-marslab.csv'

In [19]:
import re

In [24]:
for spec in ZSpec.objects.all():
    images = [
        image for image in os.listdir(output_image_dir + '/roi_browse')
        if spec.seq_id in image
    ]
    if 'pointing' in spec.filename.lower():
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        images = [image for image in images if 'pointing' + pointing in image]
    image_dict = {}
    for image in images:
        if 'rgb' in image:
            image_dict['rgb'] = image
        if 'enhanced' in image:
            image_dict['enhanced'] = image
    spec.images = str(image_dict)
    spec.clean()
    spec.save()